In [1]:
import tensorflow as tf
from skimage import io
import numpy as np
from itertools import product

In [56]:
image_file_path = r"F:\papers\FeaDistribution\spa\stego\0.2\0001.bmp"
img = io.imread(image_file_path)
img = img[:,:,0]
[height, width, channel] = np.shape(img)

ValueError: not enough values to unpack (expected 3, got 2)

In [60]:
print(img_tensor)
new_values = tf.constant([-1.0, 0.0, 1.5, 2.0, 5.0, 15, -1.0, 0.0, 1.5, 2.0, 5.0, 15], shape=[2,2,3,1])
print(new_values)
output = tf.histogram_fixed_width(values=new_values[],
                                  value_range=[0.0, 255.0],
                                  nbins=21,
                                  dtype=tf.float32)
print(output)
with tf.Session() as sess:
    output_ = sess.run(output)
    print(output_)

Tensor("Const_18:0", shape=(2000, 3000, 1), dtype=int32)
Tensor("Const_28:0", shape=(2, 2, 3, 1), dtype=float32)
Tensor("histogram_fixed_width_21:0", shape=(21,), dtype=float32)
[ 10.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]


In [61]:
labels = [1, 0, 1, 1, 1, 1]
logits = [1, 0, 1, 1, 1, 1]

labels = tf.constant(labels)
logits = tf.constant(logits)

loss = tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=labels)

with tf.Session() as sess:
    print(sess.run(loss))

ValueError: Can not squeeze dim[0], expected a dimension of 1, got 6 for 'sparse_softmax_cross_entropy_loss/remove_squeezable_dimensions/Squeeze' (op: 'Squeeze') with input shapes: [6].

In [25]:
import numpy as np
import tensorflow as tf

data = np.random.randn(1, 8, 8, 1)
data_tensor = tf.convert_to_tensor(data,
                                   dtype=tf.float32)
print(data_tensor.get_shape())

(1, 8, 8, 1)


In [24]:
files_list = ["1", "1", "1"]
files_list_tensor = tf.constant(files_list)
print(files_list_tensor)
files_list_new = tf.Session().run(files_list_tensor)
files_list_new = files_list_new.tolist()
print(files_list_new)

Tensor("Const:0", shape=(3,), dtype=string)
[b'1', b'1', b'1']


In [2]:
from glob import glob as glob
def get_files_list(file_dir, file_type="txt", start_idx=None, end_idx=None):
    """
    get the files list
    :param file_dir: file directory
    :param file_type: type of files, "*" is to get all files in this folder
    :param start_idx: start index
    :param end_idx: end index
    :return:
        file_list: a list containing full file path
    """
    pattern = "/*." + file_type
    file_list = sorted(glob(file_dir + pattern))
    total_num = len(file_list)
    if type(start_idx) is int and start_idx > total_num:
        start_idx = None
    if type(end_idx) is int and end_idx > total_num:
        end_idx = None
    file_list = file_list[start_idx:end_idx]

    return file_list

def read_data(cover_files_path, stego_files_path, start_idx=None, end_idx=None, is_shuffle=True):
    """
    read file names from the storage
    :param cover_files_path: the folder name of cover files
    :param stego_files_path: the folder name of stego files
    :param start_idx: the start index
    :param end_idx: the end index
    :param is_shuffle: whether shuffle or not (default is True)
    :return:
        cover_data_list: list of cover data
        cover_label_list: list of cover label
        stego_data_list: list of stego data
        stego_label_list: list of stego label
    """
    cover_files_list = get_files_list(cover_files_path)         # data list of cover files
    stego_files_list = get_files_list(stego_files_path)         # data list of stego files
    sample_num_cover = len(cover_files_list)                    # total pairs of samples (cover)
    sample_num_stego = len(stego_files_list)                    # total pairs of samples (stego)
    sample_num = min(sample_num_cover, sample_num_stego)        # deal with the quantity inequality of cover and stego

    cover_files_list = cover_files_list[:sample_num]            # data list of cover files
    stego_files_list = stego_files_list[:sample_num]            # data list of stego files
    cover_label_list = np.zeros(sample_num, np.int32)           # label list of cover files
    stego_label_list = np.ones(sample_num, np.int32)            # label list of stego files
    
    data_list = np.hstack((cover_files_list, stego_files_list))
    label_list = np.hstack((cover_label_list, stego_label_list))
    
    temp = np.array([data_list, label_list])
    temp_t = temp.transpose()

    if is_shuffle is True:
        np.random.shuffle(temp_t)

    if start_idx is not None and start_idx > sample_num:
        start_idx = 0
    if end_idx is not None and end_idx > sample_num:
        end_idx = sample_num

    data_list = list(temp_t[start_idx:end_idx, 0])
    label_list = list(temp_t[start_idx:end_idx, 1])

    return data_list, label_list

In [3]:
cover_files_path = r"E:\Myself\2.database\data_1000\txt\cover\128"
stego_files_path = r"E:\Myself\2.database\data_1000\txt\cover\128"
data_list, label_list = read_data(cover_files_path, stego_files_path)
print(np.shape(data_list))

(2000,)


In [4]:
def text_read_batch(text_files_list, height=200, width=576, channel=1, separator=","):
    """
    read all txt files into the memory

    :param text_files_list: text files list
    :param height: the height of QMDCT matrix
    :param width: the width of QMDCT matrix
    :param channel: the channel of QMDCT matrix
    :param separator: separator of each elements in text file

    :return:
        data: QMDCT matrixs, ndarry, shape: [files_num, height, width, 1]
    """

    files_num = len(text_files_list)
    data = np.zeros([files_num, height, width, 1], dtype=np.float32)

    i = 0
    for text_file_path in text_files_list:
        content = text_read(text_file_path, height=height, width=width, channel=channel, separator=separator)
        data[i] = content
        i = i + 1

    return data

In [ ]:
image = tf.cast(data_list[:2], tf.string)
label = tf.cast(label_list[:2], tf.int32)

print(type(image), image.get_shape())
print(type(label), label.get_shape())

# # make an input queue
input_queue = tf.train.slice_input_producer([image, label])

files_list_tensor = input_queue[0]
label = input_queue[1]

with tf.Session() as sess:
    print("", files_list_tensor.eval())

# with tf.Session() as sess:
#     a = sess.run([files_list_tensor])
# text_file_path = text_file_path.tolist()
# print(text_file_path)

# def get_batch(image, label, batch_size, capacity):
    
#     image = tf.cast(image, tf.string)
#     label = tf.cast(label, tf.int32)

#     # make an input queue
#     input_queue = tf.train.slice_input_producer([image, label])
    
#     files_list_tensor = input_queue[0]
#     label = input_queue[1]
    
#     text_file_path = tf.Session().run(files_list_tensor)
#     text_file_path = text_file_path.tolist()
#     print(text_file_path)
    
#     image = text_read_batch(text_file_path, height=200, width=576, channel=1)
    
#     image = tf.cast(image, tf.float32)
    
#     image_batch, label_batch = tf.train.batch([image, label],
#                                                 batch_size= batch_size,
#                                                 num_threads= 64, 
#                                                 capacity = capacity)

    
#     label_batch = tf.reshape(label_batch, [batch_size])
#     image_batch = tf.cast(image_batch, tf.float32)
    
#     return image_batch, label_batch

<class 'tensorflow.python.framework.ops.Tensor'> (2,)
<class 'tensorflow.python.framework.ops.Tensor'> (2,)


In [ ]:
get_batch(data_list, label_list, 128, 1)

In [4]:
import numpy as np
 
 
class DataSet(object):
 
    def __init__(self, images, labels, num_examples):
        self._images = images
        self._labels = labels
        self._epochs_completed = 0  # 完成遍历轮数
        self._index_in_epochs = 0   # 调用next_batch()函数后记住上一次位置
        self._num_examples = num_examples  # 训练样本数
 
    def next_batch(self, batch_size, fake_data=False, shuffle=True):
        start = self._index_in_epochs
 
        if self._epochs_completed == 0 and start == 0 and shuffle:
            index0 = np.arange(self._num_examples)
            print(index0)
            np.random.shuffle(index0)
            print(index0)
            self._images = np.array(self._images)[index0]
            self._labels = np.array(self._labels)[index0]
            print(self._images)
            print(self._labels)
            print("-----------------")
 
        if start + batch_size > self._num_examples:
            self._epochs_completed += 1
            rest_num_examples = self._num_examples - start
            images_rest_part = self._images[start:self._num_examples]
            labels_rest_part = self._labels[start:self._num_examples]
            if shuffle:
                index = np.arange(self._num_examples)
                np.random.shuffle(index)
                self._images = self._images[index]
                self._labels = self._labels[index]
            start = 0
            self._index_in_epochs = batch_size - rest_num_examples
            end = self._index_in_epochs
            images_new_part = self._images[start:end]
            labels_new_part = self._labels[start:end]
            return np.concatenate((images_rest_part, images_new_part), axis=0), np.concatenate(
                (labels_rest_part, labels_new_part), axis=0)
 
        else:
            self._index_in_epochs += batch_size
            end = self._index_in_epochs
            return self._images[start:end], self._labels[start:end]
 
 
if __name__ == '__main__':
    input = np.array(['a', 'b', '1', '2', '*', '3', 'c', '&', '#'])
    output = np.array(["Letter", "Letter", "Number", "Number", "Symbol", "Number", "Letter", "Symbol", "Symbol"])
    ds = DataSet(input, output, 9)
    for i in range(3):
        image_batch, label_batch = ds.next_batch(4)
        print(image_batch)
        print(label_batch)

[0 1 2 3 4 5 6 7 8]
[7 1 6 3 2 4 5 8 0]
['&' 'b' 'c' '2' '1' '*' '3' '#' 'a']
['Symbol' 'Letter' 'Letter' 'Number' 'Number' 'Symbol' 'Number' 'Symbol'
 'Letter']
-----------------
['&' 'b' 'c' '2']
['Symbol' 'Letter' 'Letter' 'Number']
['1' '*' '3' '#']
['Number' 'Symbol' 'Number' 'Symbol']
['a' 'a' '#' 'c']
['Letter' 'Letter' 'Symbol' 'Letter']


In [8]:
data = np.random.randn(128, 128, 128, 1)
data_tensor = tf.convert_to_tensor(data, dtype=tf.float32)

In [11]:
print(tf.argmax(input=data_tensor, axis=3))

Tensor("ArgMax_1:0", shape=(128, 128, 128), dtype=int64)


In [ ]:
data_sum = tf.summary

In [39]:
import tensorflow as tf
output = block_sampling_layer(data_tensor, 2)
with tf.Session() as sess:
    output_np = output.eval()
#     print(sess.run(data_tensor))
#     print(sess.run(output))
#     print("output_np", output_np)
#     print(np.shape(output_np))
    print("data: ")
    print(data[0,0,:,0])
    print(data[0,1,:,0])
    print(data[0,2,:,0])
    print(data[0,3,:,0])
    print(data[0,4,:,0])
    print(data[0,5,:,0])
    print(data[0,6,:,0])
    print(data[0,7,:,0])
    
    print("0: ", output_np[0,:,:,0])
    print("1: ", output_np[0,:,:,1])
    print("2: ", output_np[0,:,:,2])
    print("3: ", output_np[0,:,:,3])

name: block_sampling, shape: (4, 4, 4)
data: 
[-0.17161893  0.74965953 -1.02802025 -0.13199344  0.3971187   0.77830126
  0.6487124   0.59065164]
[ 0.66747562  0.63769836 -0.14392037  0.80668337  0.57702981  1.58056001
  1.13983076 -0.30065581]
[-1.31718233  0.53297881 -0.98698308  1.41755867  0.77626787 -1.59162214
 -0.32667824  0.47986914]
[ 0.08339791 -0.06889767  0.53059466  0.05671615  1.89683478  0.12357386
  1.04915767  1.10868765]
[ 0.09465779  0.32616215  1.55016374 -0.15060254 -0.17464154 -0.51680534
  0.64633626 -1.02279008]
[ 0.14910313  1.04369867  0.54898172  0.12493159 -1.77038259  0.48065937
 -0.1349778   1.53357493]
[-0.04294125  1.21117203  0.874446   -0.00136423  1.11037878 -0.67250144
  0.45949044  0.97927662]
[ 1.3988393  -1.539521    0.76896467  0.97068279  0.2023362  -0.21423867
  0.18706427  1.71685977]
0:  [[-0.17161892 -1.02802026  0.39711869  0.6487124 ]
 [-1.3171823  -0.98698306  0.77626789 -0.32667825]
 [ 0.09465779  1.55016375 -0.17464153  0.64633626]
 [-0.

In [103]:
y_train_batch1 = y_train_batch[:int(10 / 2)]
y_train_batch2 = y_train_batch[int(10 / 2):]
new_labels = list(np.array(np.array(np.array(y_train_batch1) == np.array(y_train_batch2), dtype=np.int32), dtype=np.str))
print(new_labels)

<class 'numpy.str_'>
['0', '0', '0', '1', '1']
['0', '1', '0', '1', '0']


In [105]:
labels = tf.placeholder(dtype=tf.int32, shape=(None, ), name="label")
with tf.Session() as sess:
    print(sess.run(labels, feed_dict={labels: new_labels}))

[0 1 0 1 0]


In [65]:
def moments_extraction(input_data):
    """
    this function is used for dimension unification in Jessica's paper for steganalysis of arbitrary size
    calculate the moments of feature maps -- mean, variance, maximum and minimum
    :param input_data: the input data tensor [batch_size, height, width, channels]
    :return:
    """
    
    data_max = tf.reduce_max(input_data, axis=[1, 2], keep_dims=True, name="moments_max")
    data_min = tf.reduce_min(input_data, axis=[1, 2], keep_dims=True, name="moments_min")
    data_mean, data_variance = tf.nn.moments(input_data, axes=[1, 2], keep_dims=True, name="moments_mean_var")
    
    input_data_sub_mean = tf.subtract(input_data, data_mean)
    data_variance_inverse = tf.divide(1.0, data_variance)
    data_kurtosis = tf.multiply(tf.reduce_mean(tf.pow(input_data_sub_mean, 4)), tf.pow(data_variance_inverse, 4))
    data_skwness = tf.multiply(tf.reduce_mean(tf.pow(input_data_sub_mean, 3)), tf.pow(data_variance_inverse, 3))
    
    moments = tf.concat([data_max, data_min, data_mean, data_variance, data_kurtosis, data_skwness], axis=1, name="moments")

    return moments

In [67]:
data_tensor = tf.constant(value=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18,19,20,
                                 -1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13,-14,-15,-16,-17, -18,-19,-20],
                          shape=[2,2,5,2],
                          dtype=tf.float32)

moments = moments_extraction(data_tensor)
print(moments.get_shape())


with tf.Session() as sess:
    moments_out = sess.run(moments)
    data_out = sess.run(data_tensor)
    data_variance_inverse_out = sess.run(data_variance_inverse)
    print(data_out[0][:,:,0])
    print(data_variance_inverse_out[0][:,:,0])
#     print(data_kurtosis_out[0][:,:,0])
#     print(data_skwness_out[0][:,:,0])
    print("0-0:\n", data_out[0][:,:,0])
    print("0-1:\n", data_out[0][:,:,1])
    
    print("1-0:\n", data_out[1][:,:,0])
    print("1-1:\n", data_out[1][:,:,1])
    
    print(moments_out[0])
    print(moments_out[0][:,:,0])
    

(2, 6, 1, 2)
[[  1.   3.   5.   7.   9.]
 [ 11.  13.  15.  17.  19.]]
[[ 0.03030303]]
0-0:
 [[  1.   3.   5.   7.   9.]
 [ 11.  13.  15.  17.  19.]]
0-1:
 [[  2.   4.   6.   8.  10.]
 [ 12.  14.  16.  18.  20.]]
1-0:
 [[ -1.  -3.  -5.  -7.  -9.]
 [-11. -13. -15. -17. -19.]]
1-1:
 [[ -2.  -4.  -6.  -8. -10.]
 [-12. -14. -16. -18. -20.]]
[[[  1.90000000e+01   2.00000000e+01]]

 [[  1.00000000e+00   2.00000000e+00]]

 [[  1.00000000e+01   1.10000000e+01]]

 [[  3.30000000e+01   3.30000000e+01]]

 [[  1.63063162e-03   1.63063162e-03]]

 [[  0.00000000e+00   0.00000000e+00]]]
[[  1.90000000e+01]
 [  1.00000000e+00]
 [  1.00000000e+01]
 [  3.30000000e+01]
 [  1.63063162e-03]
 [  0.00000000e+00]]


In [77]:
nbins = 5
value_range = [0.0, 5.0]
# new_values = tf.constant(value=[-1.0, 0.0, 1.5, 2.0, 5.0, 15, -1.0, 0.0, 1.5, 2.0, 5.0, 15, 
#                                -1.0, 0.0, 1.5, 2.0, 5.0, 15, -1.0, 0.0, 1.5, 2.0, 5.0, 15],
#                          dtype=tf.float32,
#                         shape=[2, 2, 6, 1])

with tf.Session() as sess:
    hist = tf.histogram_fixed_width(moments, value_range, nbins=5)
    hist_out = sess.run(hist)
    print(hist_out)

[14  1  1  0  8]


In [6]:
import librosa
import numpy as np

In [55]:
def get_mfcc_statistics(audio_data, sampling_rate=44100, n_mfcc=40):
    """
    calculate the statistics of mfcc coefficients
    :param audio_data: audio data, ndarray [length, channel]
    :param sampling_rate: sampling rate of audio data, default: 44100
    :param n_mfcc: number of mfcc, default: 40
    :return:
    """
    
    mfcc_feature = []
    for i in range(2):
        audio_data_mono = audio_data[0][i]
        mfcc_coefficients = librosa.feature.mfcc(y=audio_data_mono, sr=sampling_rate, n_mfcc=n_mfcc)

        mfcc_max = np.max(mfcc_coefficients.T, axis=0)
        mfcc_feature.append(mfcc_max)

        mfcc_min = np.min(mfcc_coefficients.T, axis=0)
        mfcc_feature.append(mfcc_min)

        mfcc_mean = np.mean(mfcc_coefficients.T, axis=0)
        mfcc_feature.append(mfcc_mean)

        mfcc_var = np.var(mfcc_coefficients.T, axis=0)
        mfcc_feature.append(mfcc_var)

        mfcc_var_inverse = np.divide(1.0, mfcc_var)
        mfcc_kurtosis = np.multiply(np.power(mfcc_mean, 4), np.power(mfcc_var_inverse, 4))
        mfcc_feature.append(mfcc_kurtosis)

        mfcc_skewness = np.multiply(np.power(mfcc_mean, 3), np.power(mfcc_var_inverse, 3))
        mfcc_feature.append(mfcc_skewness)

    mfcc_feature = np.array(mfcc_feature)

    return mfcc_feature

In [63]:
audio1_file_path = "E:/Myself/2.database/3.cover/cover_10s/128/wav10s_00001.mp3"
audio2_file_path = "E:/Myself/2.database/4.stego/EECS/EECS_B_128_W_2_H_7_ER_10/wav10s_00001.mp3"
audio1_data = librosa.load(audio1_file_path, sr=44100, mono=True, duration=1)
audio2_data = librosa.load(audio2_file_path, sr=44100, mono=False, duration=1)

print(np.shape(audio1_data))
print(np.shape(audio2_data))
      
# for i in range(2):
#     audio_data_mono = audio1_data[0][i]
#     print(np.shape(audio_data_mono))
#     mfcc_coefficients = librosa.feature.mfcc(y=audio_data_mono, sr=44100, n_mfcc=40)
#     print(np.shape(mfcc_coefficients))

(2,)
(2,)


In [43]:
print(np.shape(audio2_data[0][0]))
print(np.shape(audio2_data))

(44100,)
(2,)


In [57]:
# audio1_left_mfcc_feature = get_mfcc_statistics(audio1_data[0][0])
# audio2_left_mfcc_feature = get_mfcc_statistics(audio2_data[0][0])
# audio1_left_mfcc_feature, audio2_left_mfcc_feature

# audio1_right_mfcc_feature = get_mfcc_statistics(audio1_data[0][1])
# audio2_right_mfcc_feature = get_mfcc_statistics(audio2_data[0][1])
# audio1_right_mfcc_feature, audio2_right_mfcc_feature

audio1_left_mfcc_feature = get_mfcc_statistics(audio1_data)
audio2_left_mfcc_feature = get_mfcc_statistics(audio2_data)
audio1_left_mfcc_feature, audio2_left_mfcc_feature
print(np.shape(audio1_left_mfcc_feature))

(12, 40)


In [40]:
(audio1_left_mfcc_feature - audio2_left_mfcc_feature) / audio1_left_mfcc_feature * 100
(audio1_right_mfcc_feature - audio2_right_mfcc_feature) / audio1_right_mfcc_feature * 100

array([[ -6.96375619e-01,   3.44991943e-02,   2.30569313e+00,
          1.06476374e+00,  -8.04822502e+00,  -9.70736420e-01,
          7.98313566e+00,  -2.54826506e+00,   1.70339142e+01,
         -2.35001315e+00,  -9.91006360e+00,  -6.82365746e+00,
         -2.09166778e+01,  -1.44149134e+02,   4.89978905e+00,
          7.61643376e+00,   3.89099694e+00,   7.86649444e+00,
          1.53171715e+00,  -3.42669292e+01,  -4.13740114e+00,
          6.82122168e+00,   3.40740539e+00,  -2.30641266e+00,
         -7.00318653e+00,   3.51419688e+00,  -2.47708259e+00,
         -8.73119001e+00,  -5.46994037e+00,   2.57454906e+00,
         -1.75205876e+01,  -3.85847380e+00,  -4.12441352e+00,
         -1.66945265e+00,  -3.45783285e+01,   1.28941841e+01,
          5.30289870e+01,   3.63281393e+00,   1.35370550e+01,
         -2.89616900e+01],
       [  9.34082650e-03,  -7.71183398e-01,  -6.92618040e-01,
          1.87300685e+00,  -2.69620392e+00,  -2.53248990e+00,
         -3.10697894e-01,  -9.22248274e+01,

In [64]:
import numpy as np
import tensorflow as tf
from tensorflow.python import pywrap_tensorflow

model_file_path = r"E:\Myself\1.source_code\tf_audio_steganalysis\models\EECS_B_128_W_2_H_7_ER_10-3000"
reader = pywrap_tensorflow.NewCheckpointReader(model_file_path)
var_to_shape_map = reader.get_variable_to_shape_map()

var_to_shape_map

{'BN11/BN11/moving_mean/biased': [2],
 'BN11/BN11/moving_mean/local_step': [],
 'BN11/beta': [2],
 'BN11/beta/Adam': [2],
 'BN11/beta/Adam_1': [2],
 'BN11/gamma': [2],
 'BN11/gamma/Adam': [2],
 'BN11/gamma/Adam_1': [2],
 'BN11/moving_mean': [2],
 'BN11/moving_variance': [2],
 'BN1_3/BN1_3/moving_mean/biased': [32],
 'BN1_3/BN1_3/moving_mean/local_step': [],
 'BN1_3/beta': [32],
 'BN1_3/beta/Adam': [32],
 'BN1_3/beta/Adam_1': [32],
 'BN1_3/gamma': [32],
 'BN1_3/gamma/Adam': [32],
 'BN1_3/gamma/Adam_1': [32],
 'BN1_3/moving_mean': [32],
 'BN1_3/moving_variance': [32],
 'BN2_3/BN2_3/moving_mean/biased': [64],
 'BN2_3/BN2_3/moving_mean/local_step': [],
 'BN2_3/beta': [64],
 'BN2_3/beta/Adam': [64],
 'BN2_3/beta/Adam_1': [64],
 'BN2_3/gamma': [64],
 'BN2_3/gamma/Adam': [64],
 'BN2_3/gamma/Adam_1': [64],
 'BN2_3/moving_mean': [64],
 'BN2_3/moving_variance': [64],
 'BN3_3/BN3_3/moving_mean/biased': [128],
 'BN3_3/BN3_3/moving_mean/local_step': [],
 'BN3_3/beta': [128],
 'BN3_3/beta/Adam': [12

In [71]:
for key in var_to_shape_map:
    #print ("tensor_name",key)
    str_name = key
    # 因为模型使用Adam算法优化的，在生成的ckpt中，有Adam后缀的tensor
    if str_name.find('Adam') > -1:
        continue

    print('tensor_name:' , str_name)

    if str_name.find('/') > -1:
        names = str_name.split('/')
        # first layer name and weight, bias
        layer_name = names[0]
        layer_add_info = names[1]
    else:
        layer_name = str_name
        layer_add_info = None

#     if layer_add_info == 'weights':
#         alexnet[layer_name][0]=reader.get_tensor(key)
#     elif layer_add_info == 'biases':
#         alexnet[layer_name][1] = reader.get_tensor(key)
#     else:
#         alexnet[layer_name] = reader.get_tensor(key)

tensor_name: optimizer/beta1_power
tensor_name: BN2_3/beta
tensor_name: conv2_2/biases
tensor_name: conv1_2/biases
tensor_name: BN5_3/BN5_3/moving_mean/biased
tensor_name: BN4_3/gamma
tensor_name: BN5_3/BN5_3/moving_mean/local_step
tensor_name: BN4_3/beta
tensor_name: BN3_3/BN3_3/moving_mean/biased
tensor_name: conv1_2/weights
tensor_name: conv2_1/biases
tensor_name: BN11/gamma
tensor_name: BN1_3/gamma
tensor_name: conv6/weights
tensor_name: BN9/BN9/moving_mean/biased
tensor_name: BN5_3/beta
tensor_name: optimizer/beta2_power
tensor_name: conv6/biases
tensor_name: BN11/beta
tensor_name: BN11/BN11/moving_mean/local_step
tensor_name: BN3_3/moving_variance
tensor_name: BN1_3/beta
tensor_name: BN7/BN7/moving_mean/local_step
tensor_name: BN5_3/moving_variance
tensor_name: BN3_3/gamma
tensor_name: BN7/gamma
tensor_name: conv1_1/biases
tensor_name: BN5_3/moving_mean
tensor_name: BN2_3/BN2_3/moving_mean/local_step
tensor_name: conv5_1/biases
tensor_name: conv4_2/weights
tensor_name: conv3_2/we

In [72]:
param =[]

for key in var_to_shape_map:
    print ("tensor_name",key)
    param.append(reader.get_tensor(key))

np.save('dnnout.npy',param)

tensor_name conv5_2/weights/Adam_1
tensor_name optimizer/beta1_power
tensor_name conv10/biases/Adam_1
tensor_name BN2_3/beta
tensor_name conv2_2/biases
tensor_name conv3_1/biases/Adam
tensor_name BN2_3/gamma/Adam
tensor_name BN11/gamma/Adam_1
tensor_name BN2_3/gamma/Adam_1
tensor_name conv1_2/biases
tensor_name conv1_1/biases/Adam_1
tensor_name BN7/beta/Adam_1
tensor_name BN5_3/BN5_3/moving_mean/biased
tensor_name BN2_3/beta/Adam
tensor_name BN5_3/beta/Adam
tensor_name conv1_2/biases/Adam
tensor_name BN5_3/gamma/Adam
tensor_name BN4_3/gamma
tensor_name BN3_3/beta/Adam
tensor_name BN5_3/BN5_3/moving_mean/local_step
tensor_name conv8/biases/Adam_1
tensor_name conv1_2/weights/Adam
tensor_name BN1_3/gamma/Adam_1
tensor_name BN4_3/beta/Adam
tensor_name BN5_3/gamma/Adam_1
tensor_name conv8/biases/Adam
tensor_name conv3_1/weights/Adam
tensor_name BN4_3/beta
tensor_name conv2_2/biases/Adam
tensor_name conv8/weights/Adam
tensor_name BN3_3/BN3_3/moving_mean/biased
tensor_name BN7/beta/Adam
tenso

In [76]:
filter_diff = tf.constant(value=[1, -2, 1],
                          dtype=tf.float32,
                          shape=[3, 1, 1, 1],
                          name="diff_inter_2")

image = tf.random_normal([100, 200, 380, 1], mean=-1, stddev=4)

output = tf.nn.conv2d(input=image,
                      filter=filter_diff,
                      strides=[1, 1, 1, 1],
                      padding="VALID")
print(output.get_shape())

(100, 198, 380, 1)


In [78]:
filter_diff = tf.constant(value=[1, -2, 1],
                          dtype=tf.float32,
                          shape=[3, 1, 1, 1],
                          name="diff_inter_2")

with tf.Session() as sess:
    filter = sess.run(filter_diff)

print(type(filter_diff))
print(type(filter))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>


In [80]:
import pdfkit
pdfkit.from_url('http://google.com','out.pdf')  

OSError: No wkhtmltopdf executable found: "b''"
If this file exists please check that this process can read it. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf

In [21]:
import numpy as np
import tensorflow as tf
from skimage import io

image_file_path = r"F:\papers\FeaDistribution\spa\stego\0.4\0010.bmp"
img = io.imread(image_file_path, as_grey=True)

data = np.floor(255*img[240:250, 240:250])
print(np.shape(data))

data_dif_np = np.diff(data, 2, 0)

filter_diff = tf.constant(value=[1, -2, 1],
                          dtype=tf.float32,
                          shape=[3, 1, 1, 1],
                          name="diff_inter_2")

data_new = np.reshape(data, [1, 10, 10, 1])
data_tensor = tf.convert_to_tensor(data_new, dtype=tf.float32)
data_dif_tf = tf.nn.conv2d(input=data_tensor,
                           filter=filter_diff,
                           strides=[1, 1, 1, 1],
                           padding="VALID")

with tf.Session() as sess:
    data_dif_tf = sess.run(data_dif_tf)

data_huifu = np.reshape(data_dif_tf, [8, 10])
print("data: \n", data)
print("data_dif_np: \n", data_dif_np)
print("data_dif_tf: \n", data_huifu)

(10, 10)
data: 
 [[  70.   83.   92.   93.   96.   90.   96.   99.  102.  103.]
 [  66.   73.   82.   92.   95.   89.  100.   99.  100.  102.]
 [  57.   62.   73.   91.   95.   94.  105.  100.  100.  101.]
 [  55.   63.   69.   85.   92.   98.  104.   99.  102.  105.]
 [  61.   65.   75.   81.   95.   96.  100.  100.  105.  107.]
 [  69.   69.   85.   83.   96.   96.  103.  107.  106.  108.]
 [  72.   75.   87.   87.   94.   98.  106.  107.  110.  111.]
 [  70.   77.   84.   89.   90.   95.  102.  101.  108.  113.]
 [  73.   76.   81.   93.   89.   94.   99.  100.  110.  118.]
 [  77.   72.   85.   91.   92.   95.   99.  106.  110.  120.]]
data_dif_np: 
 [[ -5.  -1.   1.   0.   1.   6.   1.   1.   2.   0.]
 [  7.  12.   5.  -5.  -3.  -1.  -6.  -2.   2.   5.]
 [  8.   1.  10.   2.   6.  -6.  -3.   2.   1.  -2.]
 [  2.   2.   4.   6.  -2.   2.   7.   6.  -2.  -1.]
 [ -5.   2.  -8.   2.  -3.   2.   0.  -7.   3.   2.]
 [ -5.  -4.  -5.  -2.  -2.  -5.  -7.  -6.  -6.  -1.]
 [  5.  -3.   0.   